# Cross-Validation

Perform cross-validation on all the models with the public dronology dataset

In [ ]:
!pip install -U transformers datasets evaluate accelerate sentencepiece setfit trl peft bitsandbytes

# Uncomment the following code if you want to upload your model to huggingface
# !apt install git-lfs
# !git config --global user.email "YOUR_EMAIL_ADDRESS"
# !git config --global user.name "YOUR_USER_NAME"
# from huggingface_hub import notebook_login
# notebook_login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
import pandas as pd
import torch
from datasets import Dataset, DatasetDict
import os
import numpy as np

try:
    from google.colab import drive
    drive.mount('/content/gdrive')

    DATA_PATH = '/content/gdrive/MyDrive/Automated-Requirements-Classification/data/'
except:
    DATA_PATH = 'data/'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

Mounted at /content/gdrive


device(type='cuda', index=0)

In [ ]:
!cp /content/gdrive/MyDrive/Automated-Requirements-Classification/models.py /content/

## Load data

In [ ]:
def read_data(train_path: str, test_path: str) -> pd.DataFrame:
    train = pd.read_csv(DATA_PATH + train_path)
    test = pd.read_csv(DATA_PATH + test_path)

    train['text'] = train['STR.REQ']
    train['label'] = train['class']
    train = train.drop(['issueid', 'STR.REQ', 'class'], axis=1)

    test['text'] = test['STR.REQ']
    test['label'] = test['class']
    test = test.drop(['issueid', 'STR.REQ', 'class'], axis=1)

    return train, test

## DeBERTa

In [ ]:
evl_results = {
    'eval_accuracy': [],
    'eval_weighted precision': [],
    'eval_weighted recall': [],
    'eval_weighted f1': [],
    'eval_macro precision': [],
    'eval_macro recall': [],
    'eval_macro f1': []
}

In [ ]:
from models import DeBERTaModel

for i in range(1, 6):
    print('\n------------------------------')
    print(f'Fine-tuning DeBERTa on fold {i}')

    # Read data for each fold
    train_fold, test_fold = read_data(f'dronology_five_folds/fold_{i}/train_fold_{i}.csv', f'dronology_five_folds/fold_{i}/test_fold_{i}.csv')
    train = Dataset.from_pandas(train_fold)
    test = Dataset.from_pandas(test_fold)
    data = DatasetDict({'train': train, 'test': test})

    # Build model and fine-tune it on each fold
    model_name = 'microsoft/deberta-v3-large'
    deberta_model = DeBERTaModel(model_name, device)
    evl_result = deberta_model.train(data, f'fold{i}', epochs=10, push_to_hub=False)

    # Save evaluation results
    for k, v in evl_result.items():
        if k in evl_results.keys():
            evl_results[k].append(v)

    print(f'Evaluation result of fold {i}')
    print(evl_result)
    print('------------------------------\n')

    os.system(f'rm -r deberta-ReqORNot-fold{i}')


------------------------------
Fine-tuning DeBERTa on fold 1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.


Map:   0%|          | 0/303 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/76 [00:00<?, ? examples/s]

...

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,Weighted precision,Weighted recall,Weighted f1,Macro precision,Macro recall,Macro f1
1,No log,0.465754,0.746667,0.557511,0.746667,0.638372,0.373333,0.500000,0.427481
2,No log,0.816731,0.773333,0.756784,0.773333,0.761005,0.695550,0.656955,0.669775
3,No log,0.803583,0.773333,0.769513,0.773333,0.771273,0.698830,0.691729,0.695049
4,No log,1.101079,0.786667,0.767879,0.786667,0.765333,0.724432,0.648496,0.666667
5,No log,1.482790,0.760000,0.778328,0.760000,0.766984,0.693562,0.717575,0.702381
6,No log,1.374564,0.800000,0.791102,0.800000,0.794037,0.736229,0.709586,0.720497
7,0.300200,1.564739,0.786667,0.794638,0.786667,0.790061,0.720899,0.735432,0.727273
8,0.300200,1.565123,0.800000,0.796686,0.800000,0.798182,0.735380,0.726974,0.730926
9,0.300200,1.557975,0.800000,0.796686,0.800000,0.798182,0.735380,0.726974,0.730926
10,0.300200,1.585325,0.800000,0.803636,0.800000,0.801663,0.736364,0.744361,0.740125


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation result of fold 5
{'eval_loss': 1.5853251218795776, 'eval_accuracy': 0.8, 'eval_weighted precision': 0.8036363636363636, 'eval_weighted recall': 0.8, 'eval_weighted f1': 0.8016632016632016, 'eval_macro precision': 0.7363636363636363, 'eval_macro recall': 0.744360902255639, 'eval_macro f1': 0.7401247401247402, 'eval_runtime': 1.8289, 'eval_samples_per_second': 41.008, 'eval_steps_per_second': 10.389, 'epoch': 10.0}
------------------------------



In [ ]:
for k in evl_results.keys():
    print(k, np.mean(evl_results[k]))

eval_accuracy 0.8700701754385965
eval_weighted precision 0.8702830206721123
eval_weighted recall 0.8700701754385965
eval_weighted f1 0.8688699154788868
eval_macro precision 0.8410176919772272
eval_macro recall 0.8204955570745044
eval_macro f1 0.8286873063380538


## BERT-base-uncased

In [ ]:
evl_results = {
    'eval_accuracy': [],
    'eval_weighted precision': [],
    'eval_weighted recall': [],
    'eval_weighted f1': [],
    'eval_macro precision': [],
    'eval_macro recall': [],
    'eval_macro f1': []
}

In [ ]:
from models import DeBERTaModel

for i in range(1, 6):
    print('\n------------------------------')
    print(f'Fine-tuning DeBERTa on fold {i}')

    # Read data for each fold
    train_fold, test_fold = read_data(f'dronology_five_folds/fold_{i}/train_fold_{i}.csv', f'dronology_five_folds/fold_{i}/test_fold_{i}.csv')
    train = Dataset.from_pandas(train_fold)
    test = Dataset.from_pandas(test_fold)
    data = DatasetDict({'train': train, 'test': test})

    # Build model and fine-tune it on each fold
    model_name = 'bert-base-uncased'
    deberta_model = DeBERTaModel(model_name, device)
    evl_result = deberta_model.train(data, f'fold{i}', epochs=10, push_to_hub=False)

    # Save evaluation results
    for k, v in evl_result.items():
        if k in evl_results.keys():
            evl_results[k].append(v)

    print(f'Evaluation result of fold {i}')
    print(evl_result)
    print('------------------------------\n')

    os.system(f'rm -r deberta-ReqORNot-fold{i}')


------------------------------
Fine-tuning DeBERTa on fold 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/303 [00:00<?, ? examples/s]

Map:   0%|          | 0/76 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,Weighted precision,Weighted recall,Weighted f1,Macro precision,Macro recall,Macro f1
1,No log,0.477674,0.736842,0.542936,0.736842,0.625199,0.368421,0.500000,0.424242
2,No log,0.237425,0.921053,0.939271,0.921053,0.923967,0.884615,0.946429,0.906481
3,No log,0.294164,0.881579,0.918330,0.881579,0.887281,0.844828,0.919643,0.864474
4,No log,0.545217,0.881579,0.878627,0.881579,0.878323,0.860917,0.823214,0.839248
5,No log,0.411541,0.907895,0.931774,0.907895,0.911683,0.870370,0.937500,0.892199
6,No log,0.633445,0.868421,0.865295,0.868421,0.866113,0.837165,0.814286,0.824561
7,0.309000,0.671480,0.881579,0.883755,0.881579,0.882490,0.844589,0.855357,0.849703
8,0.309000,0.568861,0.894737,0.914413,0.894737,0.898623,0.855385,0.912500,0.875308
9,0.309000,0.584437,0.894737,0.899256,0.894737,0.896286,0.858586,0.880357,0.868398
10,0.309000,0.607834,0.894737,0.899256,0.894737,0.896286,0.858586,0.880357,0.868398


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/304 [00:00<?, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,Weighted precision,Weighted recall,Weighted f1,Macro precision,Macro recall,Macro f1
1,No log,0.568182,0.720000,0.705636,0.720000,0.711652,0.617055,0.603853,0.608696
2,No log,0.614589,0.746667,0.793901,0.746667,0.759854,0.696809,0.743421,0.705639
3,No log,1.310577,0.733333,0.801369,0.733333,0.749333,0.696481,0.751880,0.700000
4,No log,1.350996,0.760000,0.778328,0.760000,0.766984,0.693562,0.717575,0.702381
5,No log,1.446009,0.746667,0.751152,0.746667,0.748773,0.668182,0.673872,0.670825
6,No log,1.369638,0.773333,0.762613,0.773333,0.766576,0.696504,0.674342,0.683230
7,0.247900,1.571131,0.746667,0.742339,0.746667,0.744364,0.662281,0.656485,0.659172
8,0.247900,1.690531,0.706667,0.738667,0.706667,0.718365,0.640000,0.664474,0.646226
9,0.247900,1.616306,0.760000,0.760000,0.760000,0.760000,0.682801,0.682801,0.682801
10,0.247900,1.643727,0.760000,0.768748,0.760000,0.763818,0.687831,0.700188,0.693182


Evaluation result of fold 5
{'eval_loss': 0.6145890355110168, 'eval_accuracy': 0.7466666666666667, 'eval_weighted precision': 0.7939007092198581, 'eval_weighted recall': 0.7466666666666667, 'eval_weighted f1': 0.7598540246505543, 'eval_macro precision': 0.6968085106382979, 'eval_macro recall': 0.743421052631579, 'eval_macro f1': 0.7056393307167941, 'eval_runtime': 1.2705, 'eval_samples_per_second': 59.031, 'eval_steps_per_second': 14.954, 'epoch': 10.0}
------------------------------



In [ ]:
for k in evl_results.keys():
    print(k, np.mean(evl_results[k]))

eval_accuracy 0.8486666666666667
eval_weighted precision 0.8679757069402495
eval_weighted recall 0.8486666666666667
eval_weighted f1 0.8526781301052051
eval_macro precision 0.8064628288886638
eval_macro recall 0.8417686261107313
eval_macro f1 0.8160585839411807


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Llama

In [ ]:
evl_results = {
    'eval_accuracy': [],
    'eval_weighted precision': [],
    'eval_weighted recall': [],
    'eval_weighted f1': [],
    'eval_macro precision': [],
    'eval_macro recall': [],
    'eval_macro f1': []
}

In [ ]:
from models import LlamaModel

for i in range(1, 6):
    print('\n------------------------------')
    print(f'Fine-tuning Llama on fold {i}')

    # Read data for each fold
    train_fold, test_fold = read_data(f'dronology_five_folds/fold_{i}/train_fold_{i}.csv', f'dronology_five_folds/fold_{i}/test_fold_{i}.csv')
    train = Dataset.from_pandas(train_fold)
    test = Dataset.from_pandas(test_fold)
    data = DatasetDict({'train': train, 'test': test})

    # Build model and fine-tune it on each fold
    hf_token = 'YOUR_HF_ACCESS_TOKEN'
    model_name = 'meta-llama/Llama-2-7b-hf'
    llama_model = LlamaModel(model_name, hf_token)
    evl_result = llama_model.train(data, f'fold{i}', epochs=7, push_to_hub=False)

    # Save evaluation results
    for k, v in evl_result.items():
        if k in evl_results.keys():
            evl_results[k].append(v)

    print(f'Evaluation result of fold {i}')
    print(evl_result)
    print('------------------------------\n')

    os.system('rm -r llama-output')


------------------------------
Fine-tuning Llama on fold 1


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Weighted precision,Weighted recall,Weighted f1,Macro precision,Macro recall,Macro f1
1,No log,1.106347,0.520000,0.638293,0.520000,0.551185,0.516714,0.522086,0.487082
2,No log,0.801118,0.680000,0.656889,0.680000,0.666694,0.550000,0.542293,0.543611
3,No log,0.687267,0.680000,0.656889,0.680000,0.666694,0.550000,0.542293,0.543611
4,No log,0.651171,0.693333,0.709423,0.693333,0.700333,0.610206,0.620771,0.614008
5,No log,0.608773,0.720000,0.715166,0.720000,0.717455,0.625731,0.621241,0.623296
6,No log,0.594181,0.720000,0.715166,0.720000,0.717455,0.625731,0.621241,0.623296
7,No log,0.590194,0.720000,0.715166,0.720000,0.717455,0.625731,0.621241,0.623296


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:588: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-65e6311f-707f70c021389a6519b9250e;d5b01d78-f5cb-43a9-8035-70235335ee64)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-hf/resolve/main/config.json.
Repo model meta-llama/Llama-2-7b-hf is gated. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in meta-llama/Llama-2-7b-hf.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in meta-llama/Llama-2-7b-hf - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the fu

Evaluation result of fold 5
{'eval_loss': 0.608773410320282, 'eval_accuracy': 0.72, 'eval_weighted precision': 0.7151656920077972, 'eval_weighted recall': 0.72, 'eval_weighted f1': 0.7174551542693135, 'eval_macro precision': 0.6257309941520468, 'eval_macro recall': 0.6212406015037594, 'eval_macro f1': 0.6232958622339153, 'eval_runtime': 6.1031, 'eval_samples_per_second': 12.289, 'eval_steps_per_second': 0.492, 'epoch': 7.0}
------------------------------



In [ ]:
for k in evl_results.keys():
    print(k, np.mean(evl_results[k]))

eval_accuracy 0.7108421052631579
eval_weighted precision 0.6941365482672197
eval_weighted recall 0.7108421052631579
eval_weighted f1 0.7002403470322417
eval_macro precision 0.6099076714775444
eval_macro recall 0.5994429254955571
eval_macro f1 0.6014361127319638


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Few-Shot Learning

In [ ]:
evl_results = {
    'accuracy': [],
    'weighted precision': [],
    'weighted recall': [],
    'weighted f1': [],
    'macro precision': [],
    'macro recall': [],
    'macro f1': []
}

In [ ]:
from models import FewShotModel
import gc
from accelerate import Accelerator

accelerator = Accelerator()

for i in range(1, 6):
    print('\n------------------------------')
    print(f'Fine-tuning Few-Shot Model on fold {i}')

    # Read data for each fold
    train_fold, test_fold = read_data(f'dronology_five_folds/fold_{i}/train_fold_{i}.csv', f'dronology_five_folds/fold_{i}/test_fold_{i}.csv')
    train = Dataset.from_pandas(train_fold)
    test = Dataset.from_pandas(test_fold)
    data = DatasetDict({'train': train, 'test': test})

    # Build model and fine-tune it on each fold
    model_name = 'sentence-transformers/all-roberta-large-v1'
    few_shot_model = FewShotModel(model_name, device)
    evl_result = few_shot_model.train(data, f'fold{i}', push_to_hub=False)

    # Save evaluation results
    for k, v in evl_result.items():
        if k in evl_results.keys():
            evl_results[k].append(v)

    print(f'Evaluation result of fold {i}')
    print(evl_result)
    print('------------------------------\n')
    gc.collect()
    torch.cuda.empty_cache()
    accelerator.free_memory()


------------------------------
Fine-tuning Few-Shot Model on fold 1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/328 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 1200
  Batch size = 8
  Num epochs = 10
  Total optimization steps = 1500


Step,Training Loss


***** Running evaluation *****


Evaluation result of fold 5
{'accuracy': 0.7466666666666667, 'weighted precision': 0.7708496732026143, 'weighted recall': 0.7466666666666667, 'weighted f1': 0.7554763457219446, 'macro precision': 0.6813725490196079, 'macro recall': 0.7086466165413534, 'macro f1': 0.6902847207128886}
------------------------------



In [ ]:
for k in evl_results.keys():
    print(k, np.mean(evl_results[k]))

accuracy 0.6793684210526316
weighted precision 0.782803085311721
weighted recall 0.6793684210526316
weighted f1 0.695529375990229
macro precision 0.6735665317139212
macro recall 0.7132552973342448
macro f1 0.6520294110714173


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
